<a href="https://colab.research.google.com/github/ThamirisAdriano/gen_ia_mlet/blob/main/Redes_Generativas_Adversariais_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesta parte da aula, vamos configurar o ambiente de desenvolvimento necessário para criar e treinar Redes Generativas Adversariais (GANs). Vamos usar o Google Colab para isso, e vamos instalar as bibliotecas essenciais, como TensorFlow e Keras.

O Google Colab é uma excelente escolha, pois oferece um ambiente configurado na nuvem com suporte a GPUs, o que é muito útil para treinar modelos de deep learning como GANs.

In [3]:
# Instalando TensorFlow no Google Colab (caso necessário)
!pip install tensorflow

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

###Verificando se temos acesso a uma GPU
O que é uma GPU? Uma GPU (Graphics Processing Unit) é um tipo especial de processador que é muito bom em fazer cálculos rápidos, especialmente para tarefas que envolvem muitos dados, como treinar redes neurais.

Por que isso é importante? Quando você está treinando um modelo de inteligência artificial (como o que estamos fazendo aqui), pode demorar muito tempo se você usar apenas o processador comum do seu computador (a CPU). Usar uma GPU pode acelerar esse processo, às vezes tornando-o dezenas de vezes mais rápido.

O que o código faz? Esse pedaço de código verifica se o ambiente na nuvem, como o Google Colab tem uma GPU disponível que possamos usar. Ele simplesmente conta quantas GPUs estão disponíveis e imprime esse número. Se o resultado for zero, significa que estamos usando apenas a CPU, o que pode ser mais lento.

In [9]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


###Importando o conjunto de dados MNIST

O que é o MNIST? MNIST é um conjunto de dados muito famoso na área de inteligência artificial. Ele contém 70.000 imagens de dígitos escritos à mão (números de 0 a 9). Cada imagem é pequena, com 28 pixels de largura e 28 pixels de altura, e é em preto e branco.

Por que usar o MNIST? Como essas imagens são pequenas e simples, o MNIST é frequentemente usado para testar e treinar modelos de inteligência artificial, especialmente para quem está começando. É como um "caderno de exercícios" para aprender a fazer o computador reconhecer padrões em imagens.

O que o código faz? Aqui, estamos "baixando" essas imagens para o nosso programa. A função tf.keras.datasets.mnist.load_data() carrega as imagens do conjunto de dados MNIST. Nós dividimos esses dados em duas partes: train_images, que são as imagens que usaremos para treinar nosso modelo, e o resto (os rótulos, que identificam quais números são mostrados em cada imagem), que não usaremos por enquanto, por isso os ignoramos com _.

In [10]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()

### Normalizando as imagens para o intervalo [-1, 1]

O que é normalização? Normalização é uma técnica usada para ajustar os valores dos dados para que fiquem dentro de um intervalo específico. Isso ajuda o modelo de inteligência artificial a processar os dados de maneira mais eficiente.

Por que normalizar as imagens? As imagens do MNIST são compostas por pixels, e cada pixel tem um valor que vai de 0 a 255 (onde 0 é preto e 255 é branco). Normalizamos esses valores para que fiquem entre -1 e 1, o que facilita o trabalho da rede neural, tornando o aprendizado mais rápido e estável.

O que o código faz?

Reshape: A linha train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') ajusta as imagens para que tenham a forma correta de [28x28 pixels] e uma única camada de cor (já que são imagens em preto e branco). Além disso, converte os valores para o tipo float32, que é um formato de número mais adequado para cálculos precisos.

Normalização: A linha train_images = (train_images - 127.5) / 127.5 normaliza os valores dos pixels. Subtrai-se 127,5 de cada valor de pixel para centralizar os valores em torno de 0 e, em seguida, divide-se por 127,5 para que os valores fiquem no intervalo de -1 a 1.

In [11]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalizando para o intervalo [-1, 1]

###Configurando o buffer e o batch size

O que é buffer? O buffer, neste contexto, é uma espécie de "banco de dados temporário" onde as imagens de treinamento são armazenadas antes de serem enviadas para o modelo de inteligência artificial. Configurar o tamanho do buffer ajuda a garantir que os dados sejam processados de forma eficiente e aleatória.

O que é batch size? Batch size (tamanho do lote) refere-se ao número de imagens que o modelo processa de uma vez antes de atualizar seus "conhecimentos". Em vez de olhar para todas as 60.000 imagens de uma vez (o que seria muito lento e difícil), o modelo processa pequenos lotes de imagens, um lote de cada vez, o que torna o treinamento mais gerenciável e eficiente.

O que o código faz?
BUFFER_SIZE = 60000: Isso define o tamanho do buffer como 60.000, que é o número total de imagens de treinamento. Isso significa que todas as imagens serão armazenadas no buffer, permitindo que o modelo as processe de maneira embaralhada (aleatória) a cada ciclo de treinamento.

BATCH_SIZE = 256: Isso define o tamanho do lote como 256, o que significa que o modelo vai olhar para 256 imagens de cada vez antes de atualizar seus parâmetros. Esse tamanho é um bom equilíbrio entre velocidade e estabilidade durante o treinamento.


In [12]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

###Criando o dataset de treino

O que é um dataset de treino? Um dataset de treino é um conjunto de dados que usamos para "ensinar" um modelo de inteligência artificial. Neste caso, estamos usando as imagens do MNIST para treinar nosso modelo.

O que o código faz?

from_tensor_slices: Essa parte pega todas as imagens de treinamento e as organiza em um formato que o TensorFlow consegue entender.
shuffle: Embaralha as imagens de forma aleatória. Isso é importante porque queremos que o modelo aprenda de maneira equilibrada, sem seguir uma ordem específica das imagens.

batch: Agrupa as imagens em lotes de tamanho BATCH_SIZE (256 imagens por lote, conforme definimos anteriormente). Isso significa que o modelo processará 256 imagens de cada vez durante o treinamento, o que ajuda a acelerar o processo.

Analogia: Imagine que você tem um baralho de cartas. shuffle é como embaralhar o baralho, e batch é como dividir o baralho em pequenos grupos de cartas para facilitar o jogo.

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

###Construção da Rede Geradora

O que é a Rede Geradora? A rede geradora é uma parte da GAN que cria novas imagens a partir de ruído aleatório. Ela tenta gerar imagens que sejam tão realistas que enganem a outra parte da GAN, a rede discriminadora.

O que o código faz?

model = tf.keras.Sequential(): Estamos criando um modelo sequencial, o que significa que as camadas da rede serão empilhadas uma após a outra.
Dense: A primeira camada Dense pega um vetor de entrada de 100 números (ruído) e o transforma em um vetor maior (7x7x256). Isso é como pegar uma ideia vaga e expandi-la em algo mais concreto.

BatchNormalization: Normaliza os valores dentro da rede, ajudando o modelo a treinar de forma mais eficiente.
LeakyReLU: É uma função de ativação que permite que pequenas quantidades de informação negativa passem, o que ajuda a evitar problemas durante o treinamento.

Reshape: Esta camada reorganiza os dados para que fiquem no formato de uma imagem pequena (7x7 pixels, com 256 camadas).
Conv2DTranspose: Estas camadas aumentam a resolução da imagem, expandindo-a de 7x7 para 14x14, e depois para 28x28, até que a imagem gerada tenha o tamanho final desejado. Pense nisso como uma série de etapas para "refinar" e "ampliar" a imagem.

activation='tanh': A função tanh ajusta os valores de saída para o intervalo [-1, 1], que corresponde ao intervalo em que normalizamos as imagens de treinamento.

Analogia: Imagine que a rede geradora é como um artista que começa com uma ideia vaga (ruído) e, aos poucos, vai detalhando essa ideia até que ela se transforme em uma imagem realista.

In [14]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

generator = make_generator_model()
generator.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 12544)               │       1,254,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 12544)               │          50,176 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_10 (LeakyReLU)           │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 7, 7, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_6 (Conv2DTranspose) │ (None, 7, 7, 128)           │         819,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_7 (Conv2DTranspose) │ (None, 14, 14, 64)          │         204,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_8 (Conv2DTranspose) │ (None, 28, 28, 1)           │           1,600 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,330,944 (8.89 MB)

 Trainable params: 2,305,472 (8.79 MB)

 Non-trainable params: 25,472 (99.50 KB)

###Construção da Rede Discriminadora

Função da Rede Discriminadora: A rede discriminadora é uma parte fundamental da GAN (Generative Adversarial Network). Ela funciona como um "crítico" que avalia se uma imagem é real (do conjunto de dados de treino) ou falsa (gerada pela rede geradora). Seu objetivo é distinguir imagens reais das falsas com a maior precisão possível.



Modelo Sequencial: Assim como na rede geradora, aqui também estamos criando um modelo sequencial (Sequential), o que significa que as camadas da rede serão empilhadas uma após a outra.

In [15]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

discriminator = make_discriminator_model()
discriminator.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 7, 7, 128)           │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │           6,273 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

A Rede Discriminadora é como um crítico rigoroso que analisa imagens e decide se elas parecem reais ou falsas.

Ela usa várias camadas convolucionais para "entender" diferentes aspectos das imagens, e então, no final, decide se a imagem deve ser considerada real ou falsa com base em todos esses aspectos.

O código configura essa rede e a prepara para ser usada junto com a rede geradora no treinamento da GAN, onde as duas redes competirão para melhorar constantemente.

Analogia:
Pense na Rede Discriminadora como um avaliador de diamantes. Ele examina cada diamante (imagem) cuidadosamente, procurando por sinais que indiquem se o diamante é verdadeiro (imagem real) ou falso (imagem gerada). Ele usa várias ferramentas (camadas convolucionais) para fazer essa avaliação e, no final, dá um veredicto (0 ou 1).